In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [3]:
from dotenv import load_dotenv


load_dotenv()  # take environment variables from .env.

True

In [4]:
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [6]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

# chat = ChatHuggingFace(llm=llm, verbose=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\HP\.cache\huggingface\token
Login successful


In [7]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming Python."),
]

llm.invoke(messages)

"\nAssistant: J'adore programmer en Python."

In [8]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [49]:
TEMPLATE = """
    Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
Make sure to give your answer in the exact same format and don't add any extra line or word.
"""

In [50]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [51]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [52]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [53]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [54]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [55]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True,)

In [56]:
file_path=r"C:\Users\Hp\MCQGENERATOR\data.txt"

In [57]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [58]:
TEXT

'{\n  "1": {\n    "mcq": "What is the primary focus of biology?",\n    "options": {\n      "a": "The study of earth\'s geological features",\n      "b": "The scientific study of life",\n      "c": "The analysis of chemical reactions",\n      "d": "The exploration of outer space"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which of these is NOT a common theme in biology?",\n    "options": {\n      "a": "Evolution",\n      "b": "Energy processing",\n      "c": "Regulation of internal environment",\n      "d": "Manufacturing goods"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "What type of cells do all organisms contain?",\n    "options": {\n      "a": "Prokaryotic cells",\n      "b": "Eukaryotic cells",\n      "c": "Both prokaryotic and eukaryotic cells",\n      "d": "Neither prokaryotic nor eukaryotic cells"\n    },\n    "correct": "c"\n  },\n  "4": {\n    "mcq": "Which level of organization in biology includes studying the functioning of organs?",\n    "options"

In [59]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [60]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [61]:
try:
    # Call the chain with the input variables
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": RESPONSE_JSON  # Use the JSON string if necessary
        }
    )
    print(response)
except Exception as e:
    print(f"Error occurred: {e}")



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:{
  "1": {
    "mcq": "What is the primary focus of biology?",
    "options": {
      "a": "The study of earth's geological features",
      "b": "The scientific study of life",
      "c": "The analysis of chemical reactions",
      "d": "The exploration of outer space"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of these is NOT a common theme in biology?",
    "options": {
      "a": "Evolution",
      "b": "Energy processing",
      "c": "Regulation of internal environment",
      "d": "Manufacturing goods"
    },
    "correct": "d"
  },
  "3": {
    "mcq": "What type of cells do all organisms contain?",
    "options": {
      "a": "Prokaryotic cells",
      "b": "Eukaryotic cells",
      "c": "Both prokaryotic and eukaryotic cells",
      "d": "Neither prokaryotic nor eukaryotic cells"
    },
    "correct": "c"
  },
  "4": {
    "mcq": "Which level of organiz

In [67]:
response['review']

'\nTo help you understand how I\'ll approach this task, let me give you an example:\n\n{\n  "1": {\n    "mcq": "What is the primary focus of biology?",\n    "options": {\n      "a": "The study of earth\'s geological features",\n      "b": "The scientific study of life",\n      "c": "The analysis of chemical reactions",\n      "d": "The exploration of outer space"\n    },\n    "correct": "b"\n  },\n  "2": {\n    "mcq": "Which of these is NOT a common theme in biology?",\n    "options": {\n      "a": "Evolution",\n      "b": "Energy processing",\n      "c": "Regulation of internal environment",\n      "d": "Manufacturing goods"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "What type of cells do all organisms contain?",\n    "options": {\n      "a": "Prokaryotic cells",\n      "b": "Eukaryotic cells",\n      "c": "Both prokaryotic and eukaryotic cells",\n      "d": "Neither prokaryotic nor eukaryotic cells"\n    },\n    "correct": "c"\n  },\n  "4": {\n    "mcq": "Which level of

In [68]:
with open(file_path, 'w') as file:
    file.write(str(response['review']))

In [92]:
with open(file_path, 'r') as file:
    lines = file.readlines()

json_lines = lines[2:-1]
json_lines = ' '.join(json_lines)
json_lines


'\n {\n   "1": {\n     "mcq": "What is the primary focus of biology?",\n     "options": {\n       "a": "The study of earth\'s geological features",\n       "b": "The scientific study of life",\n       "c": "The analysis of chemical reactions",\n       "d": "The exploration of outer space"\n     },\n     "correct": "b"\n   },\n   "2": {\n     "mcq": "Which of these is NOT a common theme in biology?",\n     "options": {\n       "a": "Evolution",\n       "b": "Energy processing",\n       "c": "Regulation of internal environment",\n       "d": "Manufacturing goods"\n     },\n     "correct": "d"\n   },\n   "3": {\n     "mcq": "What type of cells do all organisms contain?",\n     "options": {\n       "a": "Prokaryotic cells",\n       "b": "Eukaryotic cells",\n       "c": "Both prokaryotic and eukaryotic cells",\n       "d": "Neither prokaryotic nor eukaryotic cells"\n     },\n     "correct": "c"\n   },\n   "4": {\n     "mcq": "Which level of organization in biology includes studying the func

In [94]:
quiz = json.loads(json_lines)

In [95]:

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [96]:
quiz_table_data

[{'MCQ': 'What is the primary focus of biology?',
  'Choices': "a: The study of earth's geological features | b: The scientific study of life | c: The analysis of chemical reactions | d: The exploration of outer space",
  'Correct': 'b'},
 {'MCQ': 'Which of these is NOT a common theme in biology?',
  'Choices': 'a: Evolution | b: Energy processing | c: Regulation of internal environment | d: Manufacturing goods',
  'Correct': 'd'},
 {'MCQ': 'What type of cells do all organisms contain?',
  'Choices': 'a: Prokaryotic cells | b: Eukaryotic cells | c: Both prokaryotic and eukaryotic cells | d: Neither prokaryotic nor eukaryotic cells',
  'Correct': 'c'},
 {'MCQ': 'Which level of organization in biology includes studying the functioning of organs?',
  'Choices': 'a: Molecular biology | b: Anatomy and physiology | c: Population evolution | d: Individual organism',
  'Correct': 'b'},
 {'MCQ': 'When did life first appear on Earth?',
  'Choices': 'a: 3.7 million years ago | b: 3.7 billion year

In [97]:
quiz=pd.DataFrame(quiz_table_data)

In [98]:
quiz.to_csv("machinelearning.csv",index=False)